# Win The Rep
Austin Moore

## Introduction

Any football play can be summarized as a cohesion of players working together to achieve a desired result. Each player is relied upon to perform a certain job in contribution to this outcome. One of the most alienating jobs for both the offense and defense is man coverage on a pass play. During these plays, a receiver and a defender use leverage and footwork to win the battle. In this paper we will introduce a physics-based model for determining the Duel Influence Probability (DIP) for both the defender and the receiver during man coverage scenarios, showcase analysis examples, and visualize the dynamics of this metric while the ball is in the air.

## Methodology

Previous Big Data Bowl submissions have explored the topic of predicting completion rates using machine learning and physics based concepts [2][3]. These models were aimed at exploring counterfactuals for use in hypothesizing expected completion percentage. The aim of this project is to utilize a model [4] to assess the probability that either the man coverage defender or the receiver will influence the outcome of a pass play after the ball it is thrown.

### Ball Trajectory

Before influence probabilities can be determined while the ball is in the air, ball position ($x_{b_{j}}$, $y_{b_{j}}$, $z_{b_{j}}$) for each timestep in the output file needs to be estimated. Kinematics equations [1] are utilized to determine the three-dimensional ball positions for all plays.

\begin{equation}
V_{b_{xy}}
= \frac{\sqrt{\bigl(x_{b_{f}} - x_{b_{i}}\bigr)^2
             + \bigl(y_{b_{f}} - y_{b_{i}}\bigr)^2}}
             {\,\Delta t\,}
\tag{1}
\end{equation}

The above equation is used to find the xy velocity of the ball. By linearly interpolating the ball flight across the $x$, $y$ plane. Given that we are estimating ball position we make the assumption that the ball initial position ($x_{b_{i}}$, $x_{b_{f}}$) are equivalent to the quarterbacks position at the beginning of the output data and is thrown from a height ($z_{b_{i}}$) of 2 yards. We make the assumption that the ball travels at a constant rate of speed until it reaches its given final position. Since this ball is often batted or caught we make the assumption that the ball is at a height of 2 yards at its landing position $z_{b_{f}}$. With these assumptions, we can model the $b_{v}$ position for any timestamp $t_{j}$

\begin{equation}
V_{b_{z}} = \frac{g\,\Delta t}{2}
\tag{2}
\end{equation}

\begin{equation}
z_{b_{j}} = 2 + V_{b_{z}}\,t_{j} \;-\; \tfrac{1}{2}\,g\,t_{j}^{2}
\tag{3}
\end{equation}

### Spot Reach Probability

The root of the DIP is a physics-based model that was first presented by W. Spearman [4] for uses in analyzing pass reception probabilities in soccer. This paper follows a similar process by using kinematics equations and minimizing for the time it will take to reach each ball spot giveninitial position $r_{0}$ and initial velocity $v_{0}$. We must also represent the physical capabilities of each player when solving for time. This is done by finding the max acceleration and velocity of each player independently using the data provided. The derived $v_{\max}$, $a_{\max}$ valued are used as limits to minimize $t$ in the equation below.

\begin{equation}
r(t)
= r_0 + v_0\,t + \tfrac{1}{2}\,a\,t^2,
\quad
v(t)\| \le v_{\max},\quad
a(t)\| \le a_{\max}.
\tag{4}
\end{equation}

Now with estimated ball trajectory and time it will take for the given player to reach each spot on the ball trajectory we can find the difference in time $T$ between the given player and the ball reach each ball spot. To model the uncertainty and provide a probability that a player will reach the ball, we apply a sigmoid softening function to $T$ [4]. The result is a probability that a given player can reach the given ball spot $P_{\text{ball}}$ at $t_j$. This paper uses the temporal uncertaintay $\sigma=0.31s$ and bounds for what is a “catchable” ball $b_z < 3\text{yd}$ [2].

\begin{equation}
P_{\text{ball}}(t) = \frac{1}{1 + e^{-\frac{T}{\frac{\sqrt{3}\sigma}{\pi}}}}
\tag{5}
\end{equation}

### Duel Influence Probability

We make the assumption that for all uncatchable ball position, $P_{ball} = 0$. With a defined interception probability for all timestamps that the ball is in the air, we can take the integral of each players probability over the ball path to get the total DIP for each player $P_{m}$.

\begin{equation}
P_m = \int_0^{t_{b_{f}}} P_{\text{ball},m}(t) \, dt
\tag{7}
\end{equation}

## Analysis

### Data Cleaning

Since we are focused on man coverage in this assessment, we distribute the influence probabilities between the target receiver and the man coverage defender. For the sake of this assessment, we will view a receiver win as a completion and a defender win as a breakup or an incompletion. Since the defender only needs to breakup the pass, we always allocate a ball spot probability to the defender first until we reach a total probability of 1. Any remaining probability density over the remaining ball flight is not utilized. DIP is assessed when the ball is thrown. To minimize the amount of noise of potential deep zones, the following analysis only includes probabilities from route concepts that should not trigger possible shell coverage as designed.

### Position Grading Example

An example use of the DIP metric is to use it for position grading of either the Quarterback, coverage defender or the target receiver. Below is an example of using the distribution of DIP values when the ball is thrown by quarterback. For application with QB grading, it will show the QBs decision making and ball placement.

<div align="center">
    <img src="qb_rep_win_prob.png" alt="QB Win Probability" width="600" height="400">
</div>

### Pass Outcome Regression

The probabilility of each player influencing a pass when the ball is released correlates with the outcome of the pass but it cannot fully predict the outcome of the pass. This is showcased with the logistic regression fit below. 

<div align="center">
    <img src="off_MWP_log_fit.png" alt="QB Win Probability" width="400" height="400">
</div>

The DIP metric determines whether or not each player can get to the ball to make a play. During the time that the ball is in the air, player positioning, aggresiveness and awareness all contribute to who will win the rep. The next section will focus on the dynamics of this metric while the ball is in the air.

## Dynamic Scenarios 

Equation 7 provides a probability from the time the ball is thrown until it reaches its final position for each player. If we reasses this probability using equation 8 at each timestamp while the ball is in the air, $t_j$, we can visualize how this probability changes over time. This will allow us to view the dynamics of the DIP over the course of the play. This will give a better understanding of how well players position themselves to make plays even when the odds are stacked against them.

\begin{equation}
P_{mj} = \int_{t_j}^{t_{b_{f}}} P_{\text{ball},m}(t) \, dt
\tag{8}
\end{equation}

### Scenario 1

Early in the fourth quarter, the Raiders go for it on fourth down to keep a win in arms reach. Jakobi Meyers creates seperation with an out move before breaking in. Levi Wallace is able to get back in stride as the ball is thrown and maintians a 50% DIP while the ball is in the air. Wallace is able to get just enough of the ball to force an incompletion and a turnover.

<div style="display: flex; justify-content: space-between;">
  <div style="flex: 1; padding-right: 10px;">
    <video controls muted autoplay loop style="width: 100%; height: auto;">
      <source src="raiders_fourth_down_inc.mp4" type="video/mp4">
    </video>
  </div>

  <div style="flex: 1; padding-left: 10px;">
    <img src="play_2023092412_3129.gif" alt="Description of GIF" style="width: 100%; height: auto;">
  </div>
</div>

### Scenario 2

The Cardinals, trailing by two in the second half, need a score to regain the lead. Michael Wilson runs a short out route to the sideline. Kyler Murray provides a quick throw to Wilson. AJ Terrell gets a great jump on the ball and peaks at a 68% DIP accelerating to make a play. Wilson, with a 30% DIP, overcomes the odds by keeping his body between the Terrell and the football and hanging on while draped in the defender. He was able to get to the goaline and setup the go-ahead TD.

<div style="display: flex; justify-content: space-between;">
  <div style="flex: 1; padding-right: 10px;">
    <video controls muted autoplay loop style="width: 100%; height: auto;">
      <source src="cardinals_td.mp4" type="video/mp4">
    </video>
  </div>

  <div style="flex: 1; padding-left: 10px;">
    <img src="play_2023111207_3086.gif" alt="Description of GIF" style="width: 100%; height: auto;">
  </div>
</div>

### Scenario 3
The Texans just picked off the Bengals late in the fourth quarter and were marching down the field. They faced a third down and attempted a play action pass to gain a first down and seal a win. Tank Dell runs an in route across the field with Cam Taylor-Britt in close pursuit. When CJ Stroud makes the throw the to Tank, the DIP is split equally between the Tank and Cam. Cam decides to abandon his man assignment and makes a play on the ball. With his cut, he improves his DIP from 50% to 65%. He is able to come down with the interception and set up a Bengals touchdown leading to a thrilling finish.

<div style="display: flex; justify-content: space-between;">
  <div style="flex: 1; padding-right: 10px;">
    <video controls muted autoplay loop style="width: 100%; height: auto;">
      <source src="texans_fourth_quarter_int.mp4" type="video/mp4">
    </video>
  </div>

  <div style="flex: 1; padding-left: 10px;">
    <img src="play_2023111202_3972.gif" alt="Description of GIF" style="width: 100%; height: auto;">
  </div>
</div>

## Future Enhancements
The provided dataset is good enough to showcase the usefullness of the DIP metric but can be improved with less assumptions. The greatest improvment would be to use this metric with actual ball trajectory data. If we know the balls trajectory, we can estimate DIP over the course of the entire trajectory and have better placed exclusions for height. This is also true for assumed max velocity and acceleration values by player. In the future, this physics based concept can be used to assess influence probabilities for all players on all passes. With clearer inputs into the model, we can properly fit the sigmoid function to limit the log loss odds.

## Conclusion
The basis for DIP is a tunable physics based model with high interpretability. Initial probabilities can tell us about the quality of quarterback decisions, the effectivess of route running and coverage skills. This is a particular application of a concept that has many uses.

## References

[1] R. Dutta, S. Ventura, R. Yurko, *NFL 3D Visualization*. Available: https://dutta.github.io/nfl3d.html

[2] R. Dutta, A.Murali, U. Ranasaria, and S. Sanjeev, *Pave the Way for NFL Passing Analytics*. Kaggle. Available: https://www.kaggle.com/code/dutta64/pave-the-way-for-nfl-passing-analytics

[3] S. Deshpande and K. Evans, *Expected Hypothetical Completion Probability*. NFL Operations. Available: https://operations.nfl.com/media/3668/big-data-bowl-deshpande_evans.pdf

[4] A. Basye, G. Dick, R. Hotovy, P. Pop and W. Spearman, *Physics-Based Modeling of Pass Probabilities in Soccer*. ResearchGate. Available: https://www.researchgate.net/publication/315166647_Physics-Based_Modeling_of_Pass_Probabilities_in_Soccer

# Win The Rep
Austin Moore

## Introduction

Football plays are coordinated efforts among 22 players pursuing a defined outcome. In man coverage on pass plays, the duel between receiver and defender determines possession. This paper presents a physics-based model that estimates Duel Influence Probability (DIP) for receiver and defender, shows how DIP evolves while the ball is in flight, and demonstrates how to apply DIP for positional grading.

## Methodology

Prior Big Data Bowl work modeled completion rates with machine learning and physics-based approaches [2][3], often for counterfactual analysis. Here, we estimate the probability that either the man-coverage defender or the receiver makes a play on the ball after it is thrown, using a probabilistic model [4].

### Ball Trajectory

We approximate ball position $(x_{b_{j}}, y_{b_{j}}, z_{b_{j}})$ at each output timestep via kinematics [1]. Horizontal velocity:

$$V_{b_{xy}} = \frac{\sqrt{(x_{b_f} - x_{b_i})^2 + (y_{b_f} - y_{b_i})^2}}{\Delta t}$$

Assumptions: initial ball position matches the quarterback at output start; release height $z_{b_i}=2$ yards; constant horizontal speed to final position with landing height $z_{b_f}=2$ yards. Vertical motion:

$$V_{b_z} = \frac{g\,\Delta t}{2}$$
$$z_{b_j} = 2 + V_{b_z} t_j - \tfrac{1}{2} g t_j^2$$

### Spot Reach Probability

Following Spearman’s reach model [4], time to each ball spot is minimized with player-specific $a_{\max}, v_{\max}$:

$$r(t) = r_0 + v_0 t + \tfrac{1}{2} a t^2, \quad \|v(t)\| \le v_{\max},\ \|a(t)\| \le a_{\max}$$

Time difference $T$ between player and ball arrival is softened with a sigmoid (temporal uncertainty $\sigma=0.31\text{s}$; catchable if $b_z < 3\text{ yd}$) [2]:

$$P_{\text{ball}}(t) = \frac{1}{1 + e^{-\frac{T}{\frac{\sqrt{3}\sigma}{\pi}}}}$$

### Duel Influence Probability

For uncatchable positions, $P_{\text{ball}}=0$. Total influence for player $m$ over flight:

$$P_m = \int_0^{t_{b_f}} P_{\text{ball},m}(t)\, dt$$

## Analysis

### Data Cleaning

We focus on man-coverage route concepts to avoid shell-coverage noise. At throw time, influence is allocated first to the defender (they only need to disrupt). Probability density beyond a total of 1 is discarded. DIP is assessed at throw.

### Position Grading Example

DIP distributions at throw can grade quarterbacks, defenders, or target receivers. For quarterbacks, DIP summarizes decision quality and ball placement.

<div align="center">
    <img src="qb_rep_win_prob.png" alt="QB Win Probability" width="600" height="400">
</div>

### Pass Outcome Regression

Pass influence probabilities correlate with outcomes but do not fully determine them. A logistic fit illustrates this relationship:

<div align="center">
    <img src="off_MWP_log_fit.png" alt="Pass Outcome Logistic Fit" width="400" height="400">
</div>

DIP measures who can reach the ball; in-flight positioning, aggressiveness, and awareness determine who actually wins the rep. We next examine DIP dynamics while the ball is airborne.

## Dynamic Scenarios

Re-evaluating DIP at each timestamp $t_j$ during flight shows how influence shifts:

$$P_{mj} = \int_{t_j}^{t_{b_f}} P_{\text{ball},m}(t)\, dt$$

### Scenario 1

Fourth down: Jakobi Meyers gains separation; Levi Wallace maintains ~50% DIP during flight and tips the ball to force an incompletion and turnover.

<div style="display: flex; justify-content: space-between;">
  <div style="flex: 1; padding-right: 10px;">
    <video controls muted autoplay loop style="width: 100%; height: auto;">
      <source src="raiders_fourth_down_inc.mp4" type="video/mp4">
    </video>
  </div>
  <div style="flex: 1; padding-left: 10px;">
    <img src="play_2023092412_3129.gif" alt="Play GIF" style="width: 100%; height: auto;">
  </div>
</div>

### Scenario 2

Trailing by two, the Cardinals hit a quick out. AJ Terrell peaks at 68% DIP, but Michael Wilson (30% DIP) shields the catch point and converts, setting up the go-ahead TD.

<div style="display: flex; justify-content: space-between;">
  <div style="flex: 1; padding-right: 10px;">
    <video controls muted autoplay loop style="width: 100%; height: auto;">
      <source src="cardinals_td.mp4" type="video/mp4">
    </video>
  </div>
  <div style="flex: 1; padding-left: 10px;">
    <img src="play_2023111207_3086.gif" alt="Play GIF" style="width: 100%; height: auto;">
  </div>
</div>

### Scenario 3

Late fourth quarter: Tank Dell and Cam Taylor-Britt split DIP (50/50) at throw. Taylor-Britt jumps the route, lifts his DIP to 65%, intercepts, and sets up a Bengals touchdown.

<div style="display: flex; justify-content: space-between;">
  <div style="flex: 1; padding-right: 10px;">
    <video controls muted autoplay loop style="width: 100%; height: auto;">
      <source src="texans_fourth_quarter_int.mp4" type="video/mp4">
    </video>
  </div>
  <div style="flex: 1; padding-left: 10px;">
    <img src="play_2023111202_3972.gif" alt="Play GIF" style="width: 100%; height: auto;">
  </div>
</div>

## Future Enhancements

- Replace assumed ball trajectories with measured 3D tracking for full-flight DIP.
- Refine player-specific $v_{\max}$ and $a_{\max}$ from tracking.
- Use full trajectories to set height-based exclusions and tune the sigmoid to minimize log loss.
- Extend DIP to all players on all passes for broader influence analytics.

## Conclusion

DIP is a tunable, interpretable physics-based metric. Initial probabilities illuminate quarterback decisions, route quality, and coverage skill. With improved inputs (true trajectories, calibrated physical limits), DIP can support richer player evaluation and decision analysis.

## References

[1] R. Dutta, S. Ventura, R. Yurko, *NFL 3D Visualization*. https://dutta.github.io/nfl3d.html

[2] R. Dutta, A. Murali, U. Ranasaria, S. Sanjeev, *Pave the Way for NFL Passing Analytics*. https://www.kaggle.com/code/dutta64/pave-the-way-for-nfl-passing-analytics

[3] S. Deshpande, K. Evans, *Expected Hypothetical Completion Probability*. https://operations.nfl.com/media/3668/big-data-bowl-deshpande_evans.pdf

[4] A. Basye, G. Dick, R. Hotovy, P. Pop, W. Spearman, *Physics-Based Modeling of Pass Probabilities in Soccer*. https://www.researchgate.net/publication/315166647_Physics-Based_Modeling_of_Pass_Probabilities_in_Soccer